In [1]:
import jieba
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

## Step1: 数据分析

In [2]:
news = pd.read_csv('F:\wiki20190720\data\sqlResult_1558435.csv',encoding='gb18030')

In [3]:
news.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [4]:
len(news)

89611

In [5]:
xinhua_news = news[news['source'] == '新华社']

In [6]:
xinhua_news

,id,author,source,content,feature,title,url
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
39,89578,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....
95,89522,NaN,新华社,新华社韩国济州6月18日电综述：亚投行第二届年会三大亮点\r\n新华社记者 耿学鹏 严蕾\r...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",综述：亚投行第二届年会三大亮点,http://china.huanqiu.com/hot/2017-06/10866439....
113,89504,NaN,新华社,新华社北京6月18日电 经军委领导批准，《军营理论热点怎么看·2017》日前印发全军。\r\...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",《军营理论热点怎么看·2017》印发全军,http://china.huanqiu.com/hot/2017-06/10866447....
305,89312,NaN,新华社,新华社兰州6月19日电（记者张钦）记者19日了解到，刚刚出台的《甘肃省网络扶贫行动的实施方案...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",甘肃力推网络扶贫：贫困山区到2020年实现4G网络全覆盖,http://china.huanqiu.com/hot/2017-06/10866626....
312,89305,NaN,新华社,新华社乌鲁木齐6月18日电（记者杜刚、王秉阳）6月18日，新疆天山天池首次举办山地马拉松比赛...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",天山天池首次举办山地马拉松赛,http://china.huanqiu.com/hot/2017-06/10866627....
314,89303,NaN,新华社,新华社北京6月19日电（记者熊丰）记者从公安部消防局获悉，19日上午6时，由公安部消防局组织...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",9省市联动开展跨区域地震救援实战拉动演练,http://china.huanqiu.com/hot/2017-06/10866630....
543,89074,刘瑞莹,新华社,暑期旅游旺季是各大旅游景区开门迎客的好时机。然而，有5家5A级景区却高兴不起来：国家旅游...,"{""type"":""旅游"",""site"":""环球"",""commentNum"":""0"",""joi...",一年内全国12家5A级景区被国家旅游局摘牌或警告,http://go.huanqiu.com/news/2016-10/9542454.html
545,89072,NaN,新华社,新华社北京6月20日电 总部设在德国波恩的国际残奥委会在当地时间19日宣布，中国残联主席...,"{""type"":""体育"",""site"":""新浪"",""commentNum"":"" 50"",""j...",张海迪获得国际残奥委会主席参选提名,http://sports.sina.com.cn/others/others/2017-0...
651,88966,王点,新华社,新华社南京4月17日电(记者潘晔)江苏拥有不少5A景区。日前，一份来自清华大学的第三方调...,"{""type"":""旅游"",""site"":""环球"",""commentNum"":""0"",""joi...",拿什么消除游客“差评”？—江苏旅游业调查,http://go.huanqiu.com/news/2017-04/10493712.html


## Step2: 数据预处理

In [7]:
news_1 = news.dropna(subset=['source', 'content'])

In [8]:
def judgement(words):
    if words["source"]=='新华社' :
            judge = 1
    else:judge = 0
    return pd.Series([judge, words['content']], index=['y', 'content'])

In [9]:
news_list = news_1.apply(judgement,axis=1)

In [10]:
news_list.head()

,y,content
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...


In [11]:
corpus = news_list.content.tolist()

In [12]:
y_value = news_list.y.values.astype(np.int)

In [13]:
y_value

array([0, 0, 0, ..., 1, 1, 1])

In [14]:
words_cut = []
mask = []
for sentence in tqdm(corpus):
    if not isinstance(sentence, str):
        mask.append(False)
        continue
    mask.append(True)
    sentence = ''.join(re.findall(r'\w+', sentence))
    words_cut.append(' '.join(jieba.cut(sentence)))
len(words_cut)

  0%|                                                | 0/87052 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wu\AppData\Local\Temp\jieba.cache
Loading model cost 1.137 seconds.
Prefix dict has been built succesfully.
100%|███████████████████████████████████| 87052/87052 [03:41<00:00, 392.70it/s]


87052

In [15]:
words_cut[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [16]:
y_value = y_value[mask]

In [17]:
y_value

array([0, 0, 0, ..., 1, 1, 1])

## Step3: 使用tfidf进行文本向量化

In [18]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b",max_features=350) 
#这里pattern是使匹配长度从2变为了1，防止中文中一些一个字的词语被忽略
X = vectorizer.fit_transform(words_cut)

In [33]:
X.shape

(87052, 350)

## Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

In [20]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [34]:
y = y_value

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=42, test_size=0.5)

In [49]:
X_train.shape,X_test.shape

((43526, 350), (43526, 350))

## Step5:在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义.

In [50]:
x_train,x_test, y_train, y_test = train_test_split(X_train, Y_train, random_state=42, test_size=0.15)

In [51]:
knc = KNeighborsClassifier() 
knc.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [52]:
knc.score(x_test,y_test)

0.9517537141981927

In [54]:
y_predict = knc.predict(x_test)

In [55]:
y_predict_probe =knc.predict_proba(x_test)

In [64]:
y_predict

array([1, 1, 1, ..., 1, 1, 1])

In [62]:
y_predict_probe

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

Precision:$$precision=\frac{TP}{TP+FP}$$

In [65]:
precision_score(y_test, y_predict)

0.9757306443343018

Recall: $$recall=\frac{TP}{TP+FN}$$

In [66]:
recall_score(y_test, y_predict)

0.9705882352941176

F1:$$F1=\frac{2PR}{P+R}$$

In [67]:
f1_score(y_test, y_predict)

0.9731526463819994

## k取3的时候

In [71]:
knc_3 = KNeighborsClassifier(n_neighbors=3) 
knc_3.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [72]:
knc_3.score(x_test,y_test)

0.9525195282585388

##  说明k取3的时候比k取5效果好

In [73]:
y_predict_k3 = knc_3.predict(x_test)
y_predict_probe_k3 =knc_3.predict_proba(x_test)

In [74]:
precision_score(y_test, y_predict_k3)

0.9775454233801851

In [75]:
recall_score(y_test, y_predict_k3)

0.9695681740904454

In [76]:
f1_score(y_test, y_predict_k3)

0.9735404574940252

## k取7的时候

In [79]:
knc_7 = KNeighborsClassifier(n_neighbors=7) 
knc_7.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [80]:
knc_7.score(x_test,y_test)

0.9500689232654311

##  说明k=7比k=5还差一点

In [81]:
y_predict_k7 = knc_7.predict(x_test)
y_predict_probe_k7 =knc_7.predict_proba(x_test)

In [82]:
precision_score(y_test, y_predict_k7)

0.9756849315068493

In [83]:
recall_score(y_test, y_predict_k7)

0.9687181230873853

In [84]:
f1_score(y_test, y_predict_k7)

0.9721890462378433

## 用k=3的时候 尝试改变leafsize

In [85]:
knc_3_leaf = KNeighborsClassifier(n_neighbors = 3,leaf_size = 40) 
knc_3_leaf.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=40, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [86]:
knc_3_leaf.score(x_test,y_test)  #似乎没有什么变化 

0.9525195282585388

In [87]:
y_predict_k3_leaf = knc_3_leaf.predict(x_test)
y_predict_probe_k3_leaf =knc_3_leaf.predict_proba(x_test)

In [88]:
precision_score(y_test,y_predict_k3_leaf)

0.9775454233801851

In [89]:
recall_score(y_test,y_predict_k3_leaf)

0.9695681740904454

In [90]:
f1_score(y_test,y_predict_k3_leaf)

0.9735404574940252

##  推测k=3的时候已经足够精确了 微调leaf_size已经没有意义了

##  改变权重类型 尝试k=7时

In [91]:
knc_weight = KNeighborsClassifier(n_neighbors=7,weights='distance')

In [92]:
knc_weight.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='distance')

In [94]:
knc_weight.score(x_test,y_test) # 略好于以uniform做权重

0.9531321795068157

##  KNN权重改为距离后 准确度提高了 不过计算过程变慢了

In [96]:
y_predict_weight = knc_weight.predict(x_test)

In [97]:
precision_score(y_test,y_predict_weight)

0.9772338240328654

In [98]:
recall_score(y_test,y_predict_weight)

0.9705882352941176

In [99]:
f1_score(y_test,y_predict_weight)

0.973899692937564

##  贝叶斯算法:

In [103]:
x_train,x_test, y_train, y_test = train_test_split(X_train.toarray(), Y_train, random_state=42, test_size=0.15)

In [104]:
Bayes = GaussianNB()
Bayes.fit(x_train, y_train)

GaussianNB(priors=None)

In [105]:
y_pred = Bayes.predict(x_test) 
y_pred_prob = Bayes.predict_proba(x_test)

In [107]:
Bayes.score(x_test,y_test)

0.9892786031551539

In [108]:
precision_score(y_test, y_pred)

0.9964127092586266

In [109]:
recall_score(y_test, y_pred)

0.9916695001700102

In [110]:
f1_score(y_test, y_pred)

0.9940354464894342

In [111]:
roc_auc_score(y_test, y_pred_prob[:, 1])

0.9829148945227285

 贝叶斯算法得分极高,比KNN的情况更精确

##  逻辑回归

In [112]:
lr = LogisticRegression() 
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [113]:
lr.score(x_test, y_test)

0.9949456272017154

In [114]:
y_predict = lr.predict(x_test)

In [115]:
precision_score(y_test, y_predict)

0.9982961322201397

In [116]:
recall_score(y_test, y_predict)

0.9960897653859232

In [117]:
f1_score(y_test, y_predict)

0.9971917283635435

目前看来得分最高的是逻辑回归算法

##  Step6: 使用 test_data 对其进行新数据的判断

逻辑回归

In [127]:
lr.score(X_test,Y_test)

0.995106373202224

贝叶斯

In [129]:
X_train, X_test, Y_train, Y_test = train_test_split(X.toarray(), y, random_state=42, test_size=0.5)

In [130]:
Bayes.score(X_test,Y_test)

0.9894086293250012

逻辑回归比贝叶斯稍强一点，在新的数据上，所以选择逻辑回归来做这次预测。step7,8已经在前面调试过了

## Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

In [131]:
y_predict = lr.predict(X)

In [133]:
news_list.head()

,y,content
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...


In [135]:
news_list["y_predict"] = y_predict

In [136]:
news_list.head()

,y,content,y_predict
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


In [146]:
news_copy_result = news_list.loc[(news_list.y == 0)& (news_list.y_predict) == 1]

In [147]:
news_copy_result.head(10)

,y,content,y_predict
63,0,2017年上海国际电影电视节，包括6月12日至16日举办的第二十三届上海电视节和6月17日至...,1
329,0,新华社北京６月２３日电（记者邢静）幸福是人们对生活最美好的期待，２０１７年哪些城市生活更幸福...,1
331,0,新华社北京６月２３日电（记者倪元锦）２１日起，京津冀迎来今年以来最强降雨。记者采访气象专家，...,1
335,0,新华社北京６月２３日电（记者 倪元锦）２１日起，京津冀迎来今年以来最强降雨。记者采访气象专家...,1
376,0,新华社合肥６月２３日电? 题：安徽不断将跨省流域补偿试点推向深入\r\n新华社记者杨丁淼\r...,1
386,0,中国经济网香港6月23日讯(记者 赵晗)22日，记者跟随“香江二十年 粤港共辉煌”香港回归2...,1
388,0,新华社重庆６月２３日电? 题：“悟空”为何跌下共享风口？——重庆首家共享单车“退市”观察\r...,1
394,0,中国经济网香港6月23日讯(记者 赵晗)22日，记者跟随“香江二十年 粤港共辉煌”香港回归2...,1
445,0,新华社贵阳６月２３日电（记者 李黔渝）贵州省政府近日下发通知，对严重影响公共安全的中学、医院...,1
541,0,刘国梁\r\n??新华社北京6月20日电记者从中国乒乓球协会获悉，国家乒乓球队管理模式将进行...,1


## Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

数据思维就是把我们要解决的实际问题，都想办法转化成向量(数据)问题。它的好处是，可以实际的把我们搜集到的已知情况很好的反映出来，这样的话，就把一个个问题转换成了数据问题，从而可以通过训练模型来达到预测效果。

机器学习思维就是直接通过将实际问题转换成数据之间的映射后，根据某些人为设定的方法，不断的修正参数，从而达到尽可能的符合实际情况的模型，并通过这个模型来预测或解决问题

## (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。